In [414]:
import pandas as pd
import numpy as np

In [415]:
#step 1: setting up the excel environment
writer = pd.ExcelWriter('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/pba50/draft forecast/upper range/control totals.xlsx', engine = 'xlsxwriter')

In [416]:
#step 2: employment resummarizing

#starts with aggregating detailed industries into 6 industry categories
#I have created a relationship file, now read it
#!!! change file location when run on other machines !!!
ind_6cat = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/industry 6 category.csv')

#then read through all employment results--currently, I am separating the two scripts, ideally
#they could be combined
#we have different treatment for e-commerce employment, part of retail sector-see "MEMO E-Commerce in UrbanSim.docx" in the documentation folder
#we may need to decide what to do for the forecast
emp = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/pba50/draft forecast/upper range/emp by ind_v2.csv')

#merge with industry mapping
emp2 = pd.merge(bttf_emp, ind_6cat, on = 'ind', how = 'left' )


In [417]:
emp2.head()

,ind,2010,2015,2020,2025,2030,2035,2040,2045,2050,ind_6cat
0,ag,25186,18990,19980,19950,20291,20099,20975,21310,21637,AGREMPN
1,art_rec,75638,111059,121698,123959,129126,106715,142050,149099,157843,HEREMPN
2,constr,171581,213857,267975,268889,292712,279855,374728,431187,472925,OTHEMPN
3,eat,261515,268083,306493,315017,326275,271870,352068,370854,392856,RETEMPN
4,ed_high,110632,73908,82807,85837,90093,88665,97519,101563,107276,HEREMPN


In [418]:
#drop unclass employment and summarize, and transpose data.
emp2_drop = emp2[:-1]
emp_6cat = emp2_drop.groupby(['ind_6cat']).sum()
emp_6cat_transpose = emp_6cat.T.reset_index()

In [419]:
emp_controls = emp_6cat_transpose.rename(columns={'index': 'year'})
emp_controls = emp_controls.reset_index(drop = True)

In [420]:
#write to the master excel file
emp_controls.to_excel(writer, sheet_name='emp_controls')

In [422]:
#step 3 population summary
#starts with aggregating pop by single age into 5 age groups
#I have created a relationship file, now read it
agegrp = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/age group.csv')

#then read through all population results
pop = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/pba50/draft forecast/upper range/population_age.csv')

In [423]:
#summarize into age groups, needed two transposes
pop_indexed= pop.set_index('year')
pop2= pop_indexed.T.reset_index()
pop2_merge = pd.merge(pop2, agegrp, left_on = 'index', right_on = 'age', how = 'inner' )
pop2_sum = pop2_merge.groupby(['age_group']).sum()
pop2_transpose = pop2_sum.T.reset_index()
pop_controls = pop2_transpose.rename(columns={'index': 'year'}).reset_index(drop = True)
pop_controls.to_excel(writer, sheet_name='pop_controls')

In [424]:
pop_controls

age_group,year,age0004,age0519,age2044,age4564,age65p
0,2010,447535.992,1321744.981,2581390.972,1931954.972,885243.991
1,2015,443101.999,1335261.998,2745529.992,2050289.988,1080685.996
2,2020,505336.971,1494840.344,2985175.109,2190421.686,1314243.966
3,2025,544644.170,1586944.969,3224921.300,2284503.157,1589443.345
4,2030,579123.002,1678144.186,3420020.560,2414075.984,1858712.319
5,2035,613190.275,1753978.755,3496760.609,2567908.464,2088433.871
6,2040,693205.844,1914269.648,3901051.481,2713726.511,2281561.262
7,2045,779356.189,2126167.116,4351297.810,2910969.353,2474238.676
8,2050,866756.985,2356258.538,4709818.062,3135351.019,2703569.996


In [426]:
# additional step for calculating share of 4 age groups using the transposed pop2
agegrp_v2 = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/4 age groups.csv')
pop2_4ag = pd.merge(pop2, agegrp_v2, left_on = 'index', right_on = 'age', how = 'inner' )
pop2_4agsum = pop2_4ag.groupby(['age_group']).sum()
pop2_4agtranspose = pop2_4agsum.T.reset_index()
pop_4ag = pop2_4agtranspose.rename(columns={'index': 'year'}).reset_index(drop = True)

In [427]:
pop_4ag.head()

age_group,year,age0019,age2034,age3564,age65p
0,2010,1769280.973,1516044.981,2997300.963,885243.991
1,2015,1778363.997,1654471.994,3141347.986,1080685.996
2,2020,2000177.315,1757669.645,3417927.150,1314243.966
3,2025,2131589.139,1867936.399,3641488.058,1589443.345
4,2030,2257267.188,2057179.572,3776916.972,1858712.319


In [428]:
pop_4ag['total'] = pop_4ag['age0019'] + pop_4ag['age2034'] + pop_4ag['age3564'] + pop_4ag['age65p']  
pop_4ag['shra1'] = pop_4ag['age0019']/pop_4ag['total']
pop_4ag['shra2'] = pop_4ag['age2034']/pop_4ag['total']
pop_4ag['shra3'] = pop_4ag['age3564']/pop_4ag['total']
pop_4ag['shra4'] = pop_4ag['age65p']/pop_4ag['total']
pop_4ag.head()

age_group,year,age0019,age2034,age3564,age65p,total,shra1,shra2,shra3,shra4
0,2010,1769280.973,1516044.981,2997300.963,885243.991,7167870.908,0.246835,0.211506,0.418158,0.123502
1,2015,1778363.997,1654471.994,3141347.986,1080685.996,7654869.973,0.232318,0.216133,0.410372,0.141176
2,2020,2000177.315,1757669.645,3417927.150,1314243.966,8490018.076,0.235592,0.207028,0.402582,0.154799
3,2025,2131589.139,1867936.399,3641488.058,1589443.345,9230456.941,0.230930,0.202367,0.394508,0.172196
4,2030,2257267.188,2057179.572,3776916.972,1858712.319,9950076.051,0.226859,0.206750,0.379587,0.186804


In [429]:
#this is the tricky part to add share of children
#first add column, then calculate value
pop_4ag['shry'] = ''
#2010 children share is given
pop_4ag.shry.iloc[0]= 0.33350388
rownum = len(pop_4ag.index)
for i in range(1, rownum):
    pop_4ag.shry.iloc[i] = pop_4ag.shry.iloc[i-1] * ( pop_4ag.shra1.iloc[i]/pop_4ag.shra1.iloc[i-1])
#then add no children share
pop_4ag['shrn'] = 1 - pop_4ag['shry']

In [430]:
pop_4ag.head()

age_group,year,age0019,age2034,age3564,age65p,total,shra1,shra2,shra3,shra4,shry,shrn
0,2010,1769280.973,1516044.981,2997300.963,885243.991,7167870.908,0.246835,0.211506,0.418158,0.123502,0.333504,0.666496
1,2015,1778363.997,1654471.994,3141347.986,1080685.996,7654869.973,0.232318,0.216133,0.410372,0.141176,0.31389,0.68611
2,2020,2000177.315,1757669.645,3417927.150,1314243.966,8490018.076,0.235592,0.207028,0.402582,0.154799,0.318313,0.681687
3,2025,2131589.139,1867936.399,3641488.058,1589443.345,9230456.941,0.230930,0.202367,0.394508,0.172196,0.312014,0.687986
4,2030,2257267.188,2057179.572,3776916.972,1858712.319,9950076.051,0.226859,0.206750,0.379587,0.186804,0.306514,0.693486


In [432]:
#step 4 household summary

#read through household results
bttf_hh = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/pba50/draft forecast/upper range/household_projections.csv')


In [433]:
#sum and write to master excel file
hh_sum = hh.groupby(['year','inc_cat_mtc_int']).sum().reset_index()
hh_controls = hh_sum.pivot(index='year', columns='inc_cat_mtc_int', values='total').reset_index()
hh_controls.to_excel(writer, sheet_name='hh_controls')

In [434]:
hh_q = hh_controls
hh_q.head()

inc_cat_mtc_int,year,q1,q2,q3,q4
0,2010,681798.000000,664489.000000,579677.000000,6.206570e+05
1,2015,765457.767680,682876.968928,595053.352054,7.034029e+05
2,2020,808489.875676,714481.759680,666631.074464,8.387623e+05
3,2025,883831.125809,765207.339273,707314.764681,9.323108e+05
4,2030,948230.040506,803846.355822,760954.009553,1.035231e+06


In [435]:
#calculate share by inc quarter
hh_q['total'] = hh_q['q1'] + hh_q['q2'] + hh_q['q3'] + hh_q['q4']  
hh_q['shrq1'] = hh_q['q1']/hh_q['total']
hh_q['shrq2'] = hh_q['q2']/hh_q['total']
hh_q['shrq3'] = hh_q['q3']/hh_q['total']
hh_q['shrq4'] = hh_q['q4']/hh_q['total']

In [436]:
hh_q.head()

inc_cat_mtc_int,year,q1,q2,q3,q4,total,shrq1,shrq2,shrq3,shrq4
0,2010,681798.000000,664489.000000,579677.000000,6.206570e+05,2.546621e+06,0.267727,0.260930,0.227626,0.243718
1,2015,765457.767680,682876.968928,595053.352054,7.034029e+05,2.746791e+06,0.278673,0.248609,0.216636,0.256082
2,2020,808489.875676,714481.759680,666631.074464,8.387623e+05,3.028365e+06,0.266972,0.235930,0.220129,0.276969
3,2025,883831.125809,765207.339273,707314.764681,9.323108e+05,3.288664e+06,0.268751,0.232680,0.215077,0.283492
4,2030,948230.040506,803846.355822,760954.009553,1.035231e+06,3.548261e+06,0.267238,0.226547,0.214458,0.291757


In [437]:
#calculate share by number of workers
hh_sumbyworker = hh.groupby(['year','hhworkers']).sum().reset_index()
hh_w = hh_sumbyworker.pivot(index='year', columns='hhworkers', values='total').reset_index()
hh_w['total'] = hh_w['w0'] + hh_w['w1'] + hh_w['w2'] + hh_w['w3'] 
hh_w['shrw0'] = hh_w['w0']/hh_w['total']
hh_w['shrw1'] = hh_w['w1']/hh_w['total']
hh_w['shrw2'] = hh_w['w2']/hh_w['total']
hh_w['shrw3'] = hh_w['w3']/hh_w['total']
hh_w.head()

hhworkers,year,w0,w1,w2,w3,total,shrw0,shrw1,shrw2,shrw3
0,2010,462021.000000,9.364120e+05,8.671980e+05,280990.000000,2.546621e+06,0.181425,0.367708,0.340529,0.110338
1,2015,504408.610937,9.802274e+05,9.444210e+05,317734.031259,2.746791e+06,0.183636,0.356863,0.343827,0.115675
2,2020,556115.621097,1.080711e+06,1.041234e+06,350304.999433,3.028365e+06,0.183636,0.356863,0.343827,0.115675
3,2025,603915.784018,1.173602e+06,1.130732e+06,380414.989804,3.288664e+06,0.183636,0.356863,0.343827,0.115675
4,2030,651587.034579,1.266242e+06,1.219988e+06,410443.776587,3.548261e+06,0.183636,0.356863,0.343827,0.115675


In [438]:
#calculate share by hh size
hh_sumbysize = hh.groupby(['year','hhsize']).sum().reset_index()
hh_s = hh_sumbysize.pivot(index='year', columns='hhsize', values='total').reset_index()
hh_s['total'] = hh_s['s1'] + hh_s['s2'] + hh_s['s3'] + hh_s['s4']  
hh_s['shrs1'] = hh_s['s1']/hh_s['total']
hh_s['shrs2'] = hh_s['s2']/hh_s['total']
hh_s['shrs3'] = hh_s['s3']/hh_s['total']
hh_s['shrs4'] = hh_s['s4']/hh_s['total']
hh_s.head()

hhsize,year,s1,s2,s3,s4,total,shrs1,shrs2,shrs3,shrs4
0,2010,677837.000000,7.925360e+05,417602.000000,658646.000000,2.546621e+06,0.266171,0.311211,0.163983,0.258635
1,2015,674525.937632,8.637369e+05,476961.509262,731566.643542,2.746791e+06,0.245569,0.314453,0.173643,0.266335
2,2020,743564.545240,9.522573e+05,525892.488003,806650.647897,3.028365e+06,0.245533,0.314446,0.173656,0.266365
3,2025,807466.319620,1.034102e+06,571099.290101,875996.696961,3.288664e+06,0.245530,0.314444,0.173657,0.266369
4,2030,871166.280060,1.115721e+06,616194.300581,945179.725555,3.548261e+06,0.245519,0.314442,0.173661,0.266378


In [439]:
#Step 5 add all hh and pop share into a dataframe
reg_dem_fcast = pd.DataFrame()
b_slice1 = hh_s[['year','shrs1','shrs2','shrs3','shrs4']].copy()
b_slice2 = hh_q[['year','shrq1','shrq2','shrq3','shrq4']].copy()
b_slice3 = hh_w[['year','shrw0','shrw1','shrw2','shrw3']].copy()
b_slice4 = 4ag[['year','shra1','shra2','shra3','shra4','shrn','shry']].copy()
reg_dem_fcast = reg_dem_fcast.append(b_slice1)
reg_dem_fcast1 = pd.merge(reg_dem_fcast,b_slice2, on = 'year', how = 'left')
reg_dem_fcast2 = pd.merge(reg_dem_fcast1,b_slice3, on = 'year', how = 'left')
reg_dem_fcast3 = pd.merge(reg_dem_fcast2,b_slice4, on = 'year', how = 'left')

In [440]:
reg_dem_fcast3.head()

,year,shrs1,shrs2,shrs3,shrs4,shrq1,shrq2,shrq3,shrq4,shrw0,shrw1,shrw2,shrw3,shra1,shra2,shra3,shra4,shrn,shry
0,2010,0.266171,0.311211,0.163983,0.258635,0.267727,0.260930,0.227626,0.243718,0.181425,0.367708,0.340529,0.110338,0.246835,0.211506,0.418158,0.123502,0.666496,0.333504
1,2015,0.245569,0.314453,0.173643,0.266335,0.278673,0.248609,0.216636,0.256082,0.183636,0.356863,0.343827,0.115675,0.232318,0.216133,0.410372,0.141176,0.68611,0.31389
2,2020,0.245533,0.314446,0.173656,0.266365,0.266972,0.235930,0.220129,0.276969,0.183636,0.356863,0.343827,0.115675,0.235592,0.207028,0.402582,0.154799,0.681687,0.318313
3,2025,0.245530,0.314444,0.173657,0.266369,0.268751,0.232680,0.215077,0.283492,0.183636,0.356863,0.343827,0.115675,0.230930,0.202367,0.394508,0.172196,0.687986,0.312014
4,2030,0.245519,0.314442,0.173661,0.266378,0.267238,0.226547,0.214458,0.291757,0.183636,0.356863,0.343827,0.115675,0.226859,0.206750,0.379587,0.186804,0.693486,0.306514


In [441]:
reg_dem_fcast3.to_excel(writer, sheet_name='reg_dem_fcast')

In [443]:
# last step, putting together regional control master file
#vacany rate
vr= pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/vacancy rate.csv')

#pop
pop_controls['totpop'] = pop_controls['age0004'] + pop_controls['age0519']+ pop_controls['age2044']+pop_controls['age4564'] + pop_controls['age65p']

#employed residents
#first set up a constant for rate of employed residents -- update if needed.
empres_r = 0.98
#then put everything together
emp_controls.year = emp_controls.year.astype(int)
emp_controls['empres'] = empres_r * (emp_controls['AGREMPN'] +emp_controls['MWTEMPN'] + emp_controls['RETEMPN'] + emp_controls['FPSEMPN'] + emp_controls['HEREMPN'] + emp_controls['OTHEMPN'])
empres = emp_controls[['year','empres']]

#five age groups
pop_5ag = pop_controls[['year','age0004','age0519','age2044','age4564','age65p']]

regional_controls = pop_controls[['year','totpop']] 
regional_controls1 = pd.merge(regional_controls, empres, on = 'year', how = 'left')
regional_controls2 = pd.merge(regional_controls1, pop_5ag, on = 'year', how = 'left')
regional_controls3 = pd.merge(regional_controls2, vr, on = 'year', how = 'left')

#add static median age -- update if needed
medianAge = [37.7,38.5,38.9,39.6,40.2,40.9,42.7,42.2,42.9]
regional_controls3.insert(loc=3, column='medianage', value=medianAge)

In [444]:
regional_controls3.head()

,year,totpop,empres,age0004,age0519,age2044,age4564,age65p,st_res_vac
0,2010,7167870.908,3389111.46,447535.992,1321744.981,2581390.972,1931954.972,885243.991,0.030000
1,2015,7654869.973,3984906.38,443101.999,1335261.998,2745529.992,2050289.988,1080685.996,0.010000
2,2020,8490018.076,4494779.80,505336.971,1494840.344,2985175.109,2190421.686,1314243.966,0.011699
3,2025,9230456.941,4643542.82,544644.170,1586944.969,3224921.300,2284503.157,1589443.345,0.013687
4,2030,9950076.051,4931031.70,579123.002,1678144.186,3420020.560,2414075.984,1858712.319,0.016013


In [445]:
regional_controls3.to_excel(writer, sheet_name='regional_controls')

In [447]:
writer.save()